In [1]:
import numpy as np

# Risk Neutral Valuation Analysis

The book claims that the rational pricing for a derivative is NOT given either by:
* a) A free-market where supply and demand come up with a price for the derivative, NOR by
* b) The expected discounted returns of the cashflows from the derivative (i.e. integral over outcomes)

Point b) is surprising to me because that is how bonds are priced, as far as I know. And I expected this to apply also to derivatives, like options. Instead, the rational price is given by Risk-Neutral-Valuation. In this notebook I will inspect what this precisely mean by reproducing and analyzing the example 2.12 of the book.

## Example 2.12 - Pricing a Call Option using Arbitrage
In the example, we price a call option of a stock $S$. The binomial model is used with two possible known outcomes: up and down.

In [2]:
S0 = 100     # initial stock price, t=0
pu = 0.6     # probability of stock going up
pd = 1 - pu  # probability of stock going down
R = 0        # risk-free rate
u = 1.2      # multiplier when the stock goes up
d = 0.8      # multiplier when the stock goes down
def S1(up: bool):  # stock price at t=1
    return S0 * (u if up else d)

The expected value of $S_1$ under objective probabilitties is $E^P[S1]$:

In [3]:
E_S1 = pu * S1(up=True) + pd * S1(up=False)  # expected value of S1
E_S1 / (1 + R)  # discounted expected value

104.0

## Call Option
Now we also compute the value of a call option $X$ on the stock.

In [4]:
K = 110
def X(S: float): return np.maximum(S - K, 0)

In [5]:
X(S1(up=True)), X(S1(up=False))

(10.0, 0.0)

The expected value of $X$ under objective probabilites $E^P[X]$ is:

In [6]:
E_X = pu * X(S1(up=True)) + pd * X(S1(up=False))
E_X / (1 + R)

6.0

The risk-neutral pricing would use different probabilites $Q$, where $p_u = p_d = 0.5$:

In [7]:
qu = ((1 + R) - d) / (u - d)  # martingale probability of the stock going up
qd = (u - (1 + R)) / (u - d)  # same for down
qu, qd

(0.5, 0.5)

And the expected value of $X$ under $Q$ is $E^Q[X]$ is:

In [8]:
EQ_X = qu * X(S1(up=True)) + qd * X(S1(up=False))
EQ_X / (1 + R)

5.0

## Replicating Portfolio
We can replicate the cashflows of the option with a replicating portfolio, made out of $x$ bonds and $y$ stocks.

In [9]:
x = (u * X(S1(up=False)) - d * X(S1(up=True))) / (u - d)
y = (X(S1(up=True)) - X(S1(up=False))) / (S0 * (u - d))
x, y

(-20.000000000000004, 0.25000000000000006)

The value $V^h_t$ of a such a portolio for $t=0, 1$ and $Z=u, d$ is:

In [10]:
def V0(num_bonds: float, num_stocks: float):
    return num_bonds + S0 * num_stocks

def V1(num_bonds: float, num_stocks: float, up: bool):
    return (1 + R) * num_bonds + S1(up) * num_stocks

print("Initial value of replicating portfolio:", V0(x, y))

Initial value of replicating portfolio: 5.0000000000000036


In [11]:
print("Final values for the replicating portfolio:", V1(x, y, up=True), V1(x, y, up=False))

Final values for the replicating portfolio: 10.000000000000004 0.0


As we can see, we can replicate the cashflows of the call option at $t=1$ with a portfolio that costs 5 at $t=0$. Therefore the rational price of the call option at $t=0$ must be 5. Otherwise, there would be an opportunity for arbitrage. Let's see how that could be done.

## Arbitrage